In [1]:
import psycopg2
import geopandas as gpd
from collections import namedtuple
from psycopg2 import sql
import shapely

In [2]:
pwfile = open("./postgis/pwfile")
pw = pwfile.readline().strip()
pwfile.close()

# had to set db auth to scram because md5 is a bad hash for security
connection = psycopg2.connect(host='192.168.1.111', port=5432, dbname='d4d', user='d4d', password='%s' % pw)
cur = connection.cursor()

In [3]:
cur.execute("SELECT column_name FROM information_schema.columns WHERE TABLE_NAME='iraq_points'")
cur.fetchall()

[('ogc_fid',),
 ('shape_orientation',),
 ('detect_type',),
 ('iids',),
 ('change_area',),
 ('weighted_centroid_lat',),
 ('target_minimum_return',),
 ('background_minimum_return',),
 ('ref_graze_angle',),
 ('processing_type',),
 ('target_maximum_return',),
 ('sec_azimuth_angle',),
 ('ref_azimuth_angle',),
 ('shape_extent_minor',),
 ('weighted_centroid_lon',),
 ('background_maximum_return',),
 ('shape_extent_major',),
 ('perimeter',),
 ('pixel_count',),
 ('timestamp',),
 ('centroid_lat',),
 ('target_total_return',),
 ('detect_strength',),
 ('eccentricity',),
 ('centroid_lon',),
 ('return_units',),
 ('name',),
 ('algorithm',),
 ('target_mean_return',),
 ('sec_graze_angle',),
 ('background_mean_return',),
 ('compactness',),
 ('wkb_geometry',)]

In [4]:
cur.execute("SELECT MAX(ST_X(wkb_geometry)) AS max_long, MAX(ST_Y(wkb_geometry)) AS max_lat FROM iraq_points")
max_long, max_lat = cur.fetchone()
cur.execute("SELECT MIN(ST_X(wkb_geometry)) AS min_long, MIN(ST_Y(wkb_geometry)) AS min_lat FROM iraq_points")
min_long, min_lat = cur.fetchone()

In [5]:
Point = namedtuple('Point', ['lat', 'long'])
Rectangle = namedtuple('Rectangle', ['top_left', 'bottom_left', 'bottom_right', 'top_right'])

def make_subrect(top_left, lat_step, long_step):
    top_right = Point(lat=top_left.lat + lat_step, long=top_left.long)
    bottom_left = Point(lat=top_left.lat, long=top_left.long + long_step)
    bottom_right = Point(lat=top_left.lat + lat_step, long=top_left.long + long_step)
    
    return Rectangle(top_left=top_left, bottom_left=bottom_left, bottom_right=bottom_right, top_right=top_right)

def make_subrects(min_lat, min_long, max_lat, max_long, lat_count=5, long_count=5):
    lat_step = (max_lat - min_lat) / lat_count
    long_step = (max_long - min_long) / long_count
    
    subrects = []
    
    for lat_row in range(lat_count):
        for long_col in range(long_count):
            top_left = Point(lat=min_lat + (lat_row*lat_step), long=min_long + (long_col * long_step))
            subrects.append(make_subrect(top_left, lat_step, long_step))
    
    return subrects

def postgis_polygon_from_rect(rect):
    return """
    ST_MakePolygon( ST_GeomFromText('LINESTRING({tl_lat} {tl_long}, {bl_lat} {bl_long}, {br_lat} {br_long}, {tr_lat} {tr_long}, {tl_lat} {tl_long})', 4326))""".strip(
    ).format(
        tl_lat=rect.top_left.lat,
        tl_long=rect.top_left.long,
        bl_lat=rect.bottom_left.lat,
        bl_long=rect.bottom_left.long,
        br_lat=rect.bottom_right.lat,
        br_long=rect.bottom_right.long,
        tr_lat=rect.top_right.lat,
        tr_long=rect.top_right.long,
    )

In [6]:
rects = make_subrects(min_lat, min_long, max_lat, max_long)
len(rects)

25

In [7]:
choice = rects[12]

In [8]:
def build_gpd_df_within_polygon(polygon):
    # NOTE: not all columns, only those I thought could have some value at all
    query = sql.SQL("""
    SELECT
    ST_AsText(ST_FlipCoordinates(wkb_geometry)),
    name,
    iids,
    timestamp,
    detect_type,
    
    pixel_count,
    change_area,
    perimeter,
    eccentricity,
    detect_strength,
    
    centroid_lat,
    centroid_lon,
    
    weighted_centroid_lat,
    weighted_centroid_lon,
    
    target_minimum_return,
    target_maximum_return,
    target_mean_return,
    target_total_return,
    
    background_minimum_return,
    background_maximum_return,
    background_mean_return,
    
    ref_graze_angle,
    ref_azimuth_angle,
    sec_graze_angle,
    sec_azimuth_angle,
    
    shape_extent_minor,
    shape_extent_major,
    return_units,
    compactness
    
    FROM iraq_points WHERE ST_Within(ST_FlipCoordinates(wkb_geometry), {polygon})
    """.strip().format(
        polygon= postgis_polygon_from_rect(choice)
    ))
    print(query.as_string(connection))
    cur.execute(query)
    
    rows_list = []
    for (geometry,name,iids,timestamp,detect_type,
         pixel_count,change_area,perimeter,eccentricity,detect_strength,
         centroid_lat,centroid_lon,
         weighted_centroid_lat,weighted_centroid_lon,
         target_minimum_return,target_maximum_return,target_mean_return,target_total_return,
         background_minimum_return,background_maximum_return,background_mean_return,
         ref_graze_angle,ref_azimuth_angle,sec_graze_angle,sec_azimuth_angle,
         shape_extent_minor,shape_extent_major,return_units,compactness) in cur:
        rows_list.append({
            "geometry": shapely.wkt.loads(geometry),
            "name": name,
            "iids": iids,
            "timestamp": timestamp,
            "detect_type": detect_type,
            "pixel_count":pixel_count,
            "change_area":change_area,
            "perimeter":perimeter,
            "eccentricity":eccentricity,
            "detect_strength":detect_strength,
            "centroid_lat":centroid_lat,
            "centroid_lon":centroid_lon,
            "weighted_centroid_lat":weighted_centroid_lat,
            "weighted_centroid_lon":weighted_centroid_lon,
            "target_minimum_return":target_minimum_return,
            "target_maximum_return":target_maximum_return,
            "target_mean_return":target_total_return,
            "target_mean_return":target_mean_return,
            "target_total_return":target_total_return,
            "background_minimum_return":background_minimum_return,
            "background_maximum_return":background_maximum_return,
            "background_mean_return":background_mean_return,
            "ref_graze_angle":ref_graze_angle,
            "ref_azimuth_angle":ref_azimuth_angle,
            "sec_graze_angle":sec_azimuth_angle,
            "sec_graze_angle":sec_graze_angle,
            "sec_azimuth_angle":sec_azimuth_angle,
            "shape_extent_minor":shape_extent_minor,
            "shape_extent_major":shape_extent_major,
            "return_units":return_units,
            "compactness":compactness
        })
        
    return gpd.GeoDataFrame(rows_list, crs="epsg:4326")

In [9]:
%%time

df = build_gpd_df_within_polygon(choice)
print("total points in polygon:", len(df))
df.head()

SELECT
    ST_AsText(ST_FlipCoordinates(wkb_geometry)),
    name,
    iids,
    timestamp,
    detect_type,
    
    pixel_count,
    change_area,
    perimeter,
    eccentricity,
    detect_strength,
    
    centroid_lat,
    centroid_lon,
    
    weighted_centroid_lat,
    weighted_centroid_lon,
    
    target_minimum_return,
    target_maximum_return,
    target_mean_return,
    target_total_return,
    
    background_minimum_return,
    background_maximum_return,
    background_mean_return,
    
    ref_graze_angle,
    ref_azimuth_angle,
    sec_graze_angle,
    sec_azimuth_angle,
    
    shape_extent_minor,
    shape_extent_major,
    return_units,
    compactness
    
    FROM iraq_points WHERE ST_Within(ST_FlipCoordinates(wkb_geometry), ST_MakePolygon( ST_GeomFromText('LINESTRING(32.25302488 42.61423528, 32.25302488 45.59361672, 34.59259722 45.59361672, 34.59259722 42.61423528, 32.25302488 42.61423528)', 4326)))
total points in polygon: 325283
CPU times: user 9.02 s, sys: 

,geometry,name,iids,timestamp,detect_type,pixel_count,change_area,perimeter,eccentricity,detect_strength,...,background_maximum_return,background_mean_return,ref_graze_angle,ref_azimuth_angle,sec_graze_angle,sec_azimuth_angle,shape_extent_minor,shape_extent_major,return_units,compactness
0,POINT (34.55462 45.58758),IW1 09 Departure #8,01Nov18S1B101586_02_IW1_09_VV_SICD_13Nov18S1B1...,2018-10-31 18:00:00-06:00,Departure,6,363.50,56.10,0.8333,-32.2,...,4.70,2.33,56.091443,100.697221,56.091927,100.698524,20.05,36.28,sqm,1.0310
1,POINT (34.57377 45.34699),IW1 09 Departure #47,01Nov18S1B101586_02_IW1_09_VV_SICD_13Nov18S1B1...,2018-10-31 18:00:00-06:00,Departure,8,484.66,146.19,0.8008,-20.5,...,12.77,8.05,56.091443,100.697221,56.091927,100.698524,36.30,60.60,sqm,2.0149
2,POINT (34.59174 45.53807),IW1 09 Departure #22,01Nov18S1B101586_02_IW1_09_VV_SICD_13Nov18S1B1...,2018-10-31 18:00:00-06:00,Departure,6,363.50,65.48,0.9345,-26.1,...,1.91,1.05,56.091443,100.697221,56.091927,100.698524,16.09,45.21,sqm,1.2033
3,POINT (34.55349 45.58035),IW1 09 Departure #27,01Nov18S1B101586_02_IW1_09_VV_SICD_13Nov18S1B1...,2018-10-31 18:00:00-06:00,Departure,7,424.08,99.39,0.9714,-24.8,...,14.89,7.10,56.091443,100.697221,56.091927,100.698524,15.44,65.00,sqm,1.5654
4,POINT (34.57564 45.34765),IW1 09 Departure #33,01Nov18S1B101586_02_IW1_09_VV_SICD_13Nov18S1B1...,2018-10-31 18:00:00-06:00,Departure,11,666.41,135.16,0.9124,-23.3,...,25.18,8.74,56.091443,100.697221,56.091927,100.698524,27.94,68.26,sqm,1.3548
